# Using llama-parse with AstraDB

### Requirements

In [1]:
# First, install the required dependencies
!pip install --quiet llama-index llama-parse astrapy

### Configuration

In [1]:
import os
import openai

from getpass import getpass

# Get all required API keys and parameters
llama_cloud_api_key = getpass("Enter your Llama Index Cloud API Key: ")
api_endpoint = input("Enter you Astra DB API Endpoint: ")
token = getpass("Enter your Astra DB Token: ")
openai_api_key = getpass("Enter your OpenAI API Key: ")

os.environ["LLAMA_CLOUD_API_KEY"] = llama_cloud_api_key
openai.api_key = openai_api_key

In [2]:
# llama-parse is async-first, running the sync code in a notebook requires the use of nest_asyncio
import nest_asyncio

nest_asyncio.apply()

### Using llama-parse to parse a PDF

In [3]:
# Grab a PDF from Arxiv for indexing
import requests 

# The URL of the file you want to download
url = "https://arxiv.org/pdf/1706.03762.pdf"
# The local path where you want to save the file
file_path = "./attention.pdf"

# Perform the HTTP request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Open the file in binary write mode and save the content
    with open(file_path, "wb") as file:
        file.write(response.content)
    print("Download complete.")
else:
    print("Error downloading the file.")

Download complete.


In [4]:
from llama_parse import LlamaParse

documents = LlamaParse(result_type="text").load_data("./attention.pdf")

Started parsing the file under job_id 9913fae4-9ab8-4f39-ad58-8f1b09d1ed80


### Storing into Astra DB

In [5]:
from llama_index.vector_stores import AstraDBVectorStore

astra_db_store = AstraDBVectorStore(
    token=token,
    api_endpoint=api_endpoint,
    collection_name="astra_v_table_llamaparse",
    embedding_dimension=1536
)

In [6]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.llms import OpenAI

node_parser = SimpleNodeParser()

nodes = node_parser.get_nodes_from_documents(documents)
print(nodes)

[TextNode(id_='fece6211-16cf-411d-b31a-e0ed7073acb6', embedding=None, metadata={'file_path': './attention.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='6c9edaa9-35e7-4b06-b470-27016d6d4a88', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': './attention.pdf'}, hash='4d5a7515003ce9a74923078283da5a554b901cd586b9211907aba1c1f0fd2a60'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='71963830-f22c-44ca-aaee-81614feedf86', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='47f7ae41f34f9f853038324dac7e041487e641ee10cdf9b6a8d226537f3eecd1')}, text='Provided proper attribution is provided, Google hereby grants permission to\n                          reproduce the tables and figures in this paper solely for use in journalistic or\n                                                                   scholarly works.\n                                                   Attention Is A

In [7]:
from llama_index import (
    OpenAIEmbedding,
    VectorStoreIndex,
    StorageContext,
    ServiceContext,
)

storage_context = StorageContext.from_defaults(vector_store=astra_db_store)

service_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-4"), 
    embed_model=OpenAIEmbedding(), 
    chunk_size=512,
)

index = VectorStoreIndex(nodes=nodes, storage_context=storage_context)

### Simple RAG Example

In [8]:
query_engine = index.as_query_engine(similarity_top_k=15, service_context=service_context)

In [9]:
query = "What is Multi-Head Attention also known as?"

response_1 = query_engine.query(query)
print("\n***********New LlamaParse+ Basic Query Engine***********")
print(response_1)


***********New LlamaParse+ Basic Query Engine***********
Multi-Head Attention is also referred to as multiple attention layers operating simultaneously.


In [10]:
# Query fails to be answered due to lack of context in Astra DB
query = "What is the color of the sky?"

response_1 = query_engine.query(query)
print("\n***********New LlamaParse+ Basic Query Engine***********")
print(response_1)


***********New LlamaParse+ Basic Query Engine***********
The context does not provide information about the color of the sky.
